Useful resources:
* https://tomassetti.me/parsing-in-python/
* https://github.com/lark-parser/lark/blob/master/docs/lark_cheatsheet.pdf
* https://github.com/lark-parser/lark/blob/9f666a74595ff8ac0f550abbb687a517fe5d495d/lark/grammars/common.lark
* https://gitlab.cern.ch/lhcb-datapkg/Gen/DecFiles/tree/master/dkfiles
* https://github.com/PMunkes/evtgen/blob/master/DECAY.DEC


In [1]:
from particle import Particle
from particle.particle.particle import  ParticleNotFound
from decaylanguage import data

In [2]:
with data.open_text(data, 'DECAY_LHCB.DEC') as f:
    txt = f.read()

In [3]:
with data.open_text(data, 'decfile.lark') as f:
    grammar = f.read()

In [4]:
from decaylanguage.dec import dec
from lark import Lark, Transformer, Tree

/Users/henryiii/git/software/decaylanguage/decaylanguage/decay/decay.py:22: UserWarning: Graphviz is not installed. Line display not available.
  warnings.warn("Graphvis not installed. Line display not available.")


ModuleNotFoundError: No module named 'lark'

In [5]:
l = Lark(grammar, parser='lalr', lexer='standard') # , transformer = TreeToDec())

In [6]:
%%time
parsed = l.parse(txt)
bool(parsed)

CPU times: user 1.07 s, sys: 18.5 ms, total: 1.09 s
Wall time: 1.09 s


In [7]:
transformed = dec.TreeToDec().transform(parsed)

In [8]:
define = dec.define(transformed)
pythia_def = dec.pythia_def(transformed)
alias = dec.alias(transformed)
chargeconj = dec.chargeconj(transformed)
global_photos = dec.global_photos(transformed)
decay = dec.decay(transformed)
cdecay = dec.cdecay(transformed)
setlspw = dec.setlspw(transformed)

In [9]:
for item in pythia_def:
    print(item[0], ":", item[1], "=", item[2])

ParticleDecays : mixB = off
Init : showChangedSettings = off
Init : showChangedParticleData = off
Next : numberShowEvent = 0


In [10]:
missing = set()
keyerrs = set()

class TreeToDec2(Transformer):
    def __init__(self, alias_dict):
        self.alias_dict = alias_dict
    def particle(self, items):
        label, = items
        if label in self.alias_dict:
            label = self.alias_dict[label]
        try:
            return Particle.from_string(str(label))
        except ParticleNotFound:
            missing.add(str(label))
            return str(label)
        except KeyError:
            keyerrs.add(str(label))
            return str(label)

In [11]:
%%time
labelled = TreeToDec2(alias).transform(decay)

CPU times: user 50.3 s, sys: 381 ms, total: 50.7 s
Wall time: 51.1 s


In [12]:
missing

{"B'_1+",
 "B'_1-",
 "B'_10",
 "B'_c1+",
 "B'_c1-",
 "B'_s10",
 'B_0*+',
 'B_0*-',
 'B_0*0',
 'B_1+',
 'B_1-',
 'B_10',
 'B_2*+',
 'B_2*-',
 'B_2*0',
 'B_c*+',
 'B_c*-',
 'B_c0*+',
 'B_c0*-',
 'B_c1+',
 'B_c1-',
 'B_c2*+',
 'B_c2*-',
 'B_s0*0',
 'B_s10',
 'B_s2*0',
 "D'_1+",
 "D'_1-",
 "D'_10",
 "D'_s1+",
 "D'_s1-",
 'D(2S)+',
 'D(2S)-',
 'D(2S)0',
 'D*(2S)+',
 'D*(2S)-',
 'D*(2S)0',
 'D_0*+',
 'D_0*-',
 'D_0*0',
 'D_1+',
 'D_1-',
 'D_10',
 'D_2*+',
 'D_2*-',
 'D_2*0',
 'D_s*+',
 'D_s*-',
 'D_s0*+',
 'D_s0*-',
 'D_s1+',
 'D_s1-',
 'D_s2*+',
 'D_s2*-',
 'Delta(1600)+',
 'Delta(1600)++',
 'Delta(1600)-',
 'Delta(1600)0',
 'Delta(1620)+',
 'Delta(1620)++',
 'Delta(1620)-',
 'Delta(1620)0',
 'Delta(1700)+',
 'Delta(1700)++',
 'Delta(1700)-',
 'Delta(1700)0',
 'Delta(1900)+',
 'Delta(1900)++',
 'Delta(1900)-',
 'Delta(1900)0',
 'Delta(1905)+',
 'Delta(1905)++',
 'Delta(1905)-',
 'Delta(1905)0',
 'Delta(1910)+',
 'Delta(1910)++',
 'Delta(1910)-',
 'Delta(1910)0',
 'Delta(1920)+',
 'Delta(192

In [13]:
keyerrs

{'a_0+', 'a_0-'}

In [14]:
print(labelled.children[0].pretty())

decay
  Upsilon(4S)0
  decayline
    0.515122645
    B+
    B-
    model	VSS
  decayline
    0.483122645
    B0
    B~0
    model
      VSS
      model_options
        _BMIX
        dm
  decayline
    1.5583e-05
    e+
    e-
    PhotosEnum.yes
    model	VLL
  decayline
    1.5766e-05
    mu+
    mu-
    PhotosEnum.yes
    model	VLL
  decayline
    1.5766e-05
    tau+
    tau-
    PhotosEnum.yes
    model	VLL
  decayline
    8.4099e-05
    Upsilon(2S)0
    pi+
    pi-
    model
      VVP
      model_options	IPI
  decayline
    4.4342e-05
    Upsilon(2S)0
    pi0
    pi0
    model
      VVP
      model_options	IPI
  decayline
    8.0123e-05
    Upsilon(1S)0
    pi+
    pi-
    model
      VVP
      model_options	IPI
  decayline
    4.4342e-05
    Upsilon(1S)0
    pi0
    pi0
    model
      VVP
      model_options	IPI
  decayline
    0.000194392
    Upsilon(1S)0
    eta0
    model
      PARTWAVE
      model_options
        0.0
        0.0
        1.0
        0.0
        0.0
        0.0
